# <center> NN model with triplet loss

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os 
import PIL
import glob
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from source.network import NNClassifier

from source.helpme import get_gesture_dataset, load_imgs_from_folder, show_image, calculate_pad, show_history
from source.helpme import create_loader

#### Load the dataset

In [ ]:
X, y = get_gesture_dataset(size=(64, 64), shuffle=True, gray_scale=False)

X.shape, y.shape

#### Define transforms that will be applied to images

In [ ]:
trs = transforms.Compose([
    transforms.ColorJitter(brightness=1, contrast=1, saturation=0.5, hue=0),
    transforms.ToTensor()
])

#### Create data loaders

In [ ]:
# split the dataset in three parts: train, valid, test
train_size = 1400
valid_size = 250
test_size = len(X) - train_size - valid_size
print('train size =', train_size)
print('valid size =', valid_size)
print('test size =', test_size)

X_train = X[ : train_size]
y_train = y[ : train_size]

X_valid = X[train_size : train_size+valid_size]
y_valid = y[train_size : train_size+valid_size]

X_test = X[train_size+valid_size : ]
y_test = y[train_size+valid_size : ]

In [ ]:
# create three data loaders: train, valid, test
train_loader = create_loader(X_train, y_train, batch_size=512, num_workers=1, shuffle=False, trs=trs)
valid_loader = create_loader(X_valid, y_valid, batch_size=len(X_valid), num_workers=1, shuffle=False, trs=None)
test_loader = create_loader(X_test, y_test, batch_size=len(X_test), num_workers=1, shuffle=False, trs=None)

In [ ]:
# show random image with applied transforms
for x_batch, y_batch in train_loader:
    show_image(x_batch[3])
    break

# Convolutional neural network

In [ ]:
class Softmax_layer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        e = torch.exp(x - x.max(1, True)[0] )
        summ = e.sum(1, True)[0]
        return e / summ
    
class Flatten(torch.nn.Module):
    def forward(self, x):
        N = x.shape[0]
        return x.view(N, -1)

In [ ]:
calculate_pad(input_size = 64, kernel_size = 3, stride = 2, output_size = 32)

In [ ]:
conv_net = torch.nn.Sequential(torch.nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(8),
                               torch.nn.Dropout2d(0.01),
                               # 32x32x8
                               
                               torch.nn.Conv2d(8, 16, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(16),
                               torch.nn.Dropout2d(0.01),
                               # 16x16x16
                               
                               torch.nn.Conv2d(16, 32, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(32),
                               torch.nn.Dropout2d(0.01),
                               # 8x8x32
                               
                               torch.nn.Conv2d(32, 64, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(64),
                               torch.nn.Dropout2d(0.01),
                               # 4x4x64
                               
                               torch.nn.Conv2d(64, 128, 3, stride=2, padding=1),
                               torch.nn.ReLU(),
                               torch.nn.BatchNorm2d(128),
                               torch.nn.Dropout2d(0.01),
                               # 2x2x128
                               
                               Flatten(),
                               torch.nn.Linear(512, 10),
                               Softmax_layer())

In [ ]:
model = NNClassifier(conv_net, lr=1e-3, device='cpu')

In [ ]:
model.fit_loader(train_loader, valid_loader, epochs=15, log_every_epoch=5)

In [ ]:
show_history(model.train_history, model.valid_history, width=2, fontsize=10)

In [ ]:
model.evaluate_score(X_test, y_test)

# Testing on real data

Перед тем как перейти к тестированию на реальных данных, надо положить картинки в любом формате в папку source/real_data. Все что там находится можно удалить.

In [ ]:
imgs = load_imgs_from_folder(os.path.join('source', 'real_data'), size=(64,64))

In [ ]:
imgs.shape

In [ ]:
show_image(imgs[12])

In [ ]:
model.predict(imgs)

#### Save the model

In [ ]:
torch.save(conv_net.state_dict(), './my_network.pt')